In [1]:
import numpy as np
import matplotlib
import netCDF4
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy as cart
import matplotlib.ticker as mticker
from matplotlib import pyplot as plt, animation
from cartopy.mpl.gridliner import LongitudeFormatter, LatitudeFormatter
%matplotlib inline
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import climtas.blocked
import climtas.io
import glob
import dask
import bottleneck
import dask.diagnostics
dask.diagnostics.ProgressBar().register()

In [2]:
# Open all year files for hourly mean surface latent heat flux from ERA-5 and specify dask chunks

ds = xr.open_mfdataset("/g/data/rt52/era5/single-levels/reanalysis/mslhf/*/*.nc",  combine='nested', concat_dim='time', chunks={'longitude':93*2,'latitude':91*2})
ds

,Array,Chunk
Bytes,1.53 TB,100.74 MB
Shape,"(368177, 721, 1440)","(744, 182, 186)"
Count,32760 Tasks,16128 Chunks
Type,float32,numpy.ndarray


In [4]:
#Select region I'm interested in for baseline period
ds = ds.sel(time = slice('1980-01-01 00:00:00','2019-12-31 23:00:00'))
mslhf_hourly = ds.mslhf.sel(latitude = slice(70, -70), longitude =slice(-160,20))
mslhf_hourly

,Array,Chunk
Bytes,567.31 GB,100.74 MB
Shape,"(350640, 561, 721)","(744, 182, 186)"
Count,57720 Tasks,9600 Chunks
Type,float32,numpy.ndarray


In [6]:
#resamle hourly data into daily data
mslhf_daily = climtas.blocked.blocked_resample(mslhf_hourly, time=24).mean()
mslhf_daily 

,Array,Chunk
Bytes,23.64 GB,4.20 MB
Shape,"(14610, 561, 721)","(31, 182, 186)"
Count,67320 Tasks,9600 Chunks
Type,float32,numpy.ndarray


In [19]:
#calculate a 15-day rolling mean on the timeseries
mslhf_15d = (mslhf_daily).rolling(time=15, center=True).mean()
#calculate the climatology using the 15d rolling mean
mslhf_climo = mslhf_15d.groupby('time.dayofyear').mean('time', keep_attrs=True)
mslhf_climo= mslhf_climo.rename('climo_mslhf')
#calucalte the anomalies for each day
mslhf_anom = mslhf_daily.groupby('time.dayofyear')  - mslhf_climo
mslhf_anom = mslhf_anom.rename('anom_mslhf')

In [20]:
#merge the two variables mslhf_climo and mslhf_anom into one dataset
mslhf = xr.merge([mslhf_climo,mslhf_anom], combine_attrs='override')

In [21]:
mslhf.climo_mslhf

,Array,Chunk
Bytes,1.18 GB,270.82 kB
Shape,"(366, 561, 721)","(1, 182, 186)"
Count,1011280 Tasks,7320 Chunks
Type,float64,numpy.ndarray


In [22]:
mslhf.anom_mslhf

,Array,Chunk
Bytes,47.28 GB,270.82 kB
Shape,"(14610, 561, 721)","(1, 182, 186)"
Count,2194720 Tasks,292200 Chunks
Type,float64,numpy.ndarray


In [23]:
#How many GB is the mslhf dataset
mslhf.nbytes / 1024 ** 3

45.1320638731122

In [13]:
#save mslhf into a nc file
climtas.io.to_netcdf_throttled(mslhf, '/g/data/e14/cr3027/anom_MSLHF.70to-70.-160to20.1980.2019.nc')